In [45]:
import json
import numpy as np
import pandas as pd
import os
import glob
import re
import matplotlib.pyplot as plt
import random


from scipy.spatial.distance import cdist
from scipy.optimize import linear_sum_assignment


import pyclustering
from pyclustering.cluster.kmedoids import kmedoids
from pyclustering.utils.metric import distance_metric, type_metric
from sklearn.manifold import MDS

seed = 42


In [46]:

def load_data(path):
    with open(path, 'r') as file:
        data = json.load(file)

    parsed_data = {}
    for key, value in data.items():
        # Check if the value is a list
        if isinstance(value, list):
            try:
                # Attempt to convert to a NumPy array
                subfield = np.array(value)

                if subfield.ndim == 1:  # 1D array
                    parsed_data[key] = pd.Series(value)
                elif subfield.ndim == 2:  # 2D array
                    parsed_data[key] = pd.DataFrame(value)
                else:  # Higher-dimensional array
                    parsed_data[key] = {i: pd.DataFrame(subarray) for i, subarray in enumerate(subfield)}
            except ValueError:
                # If conversion fails, handle as a dictionary of DataFrames
                parsed_data[key] = {i: pd.DataFrame(subarray) for i, subarray in enumerate(value) if isinstance(subarray, list)}
        else:
            # For non-list data, store it as-is
            parsed_data[key] = value

    return parsed_data

In [47]:
# From sig_profiler_extractor
def evaluation(true_sigs, est_sigs, cutoff=None, dist="cos", verbose=False):
    if true_sigs.shape[1] >= est_sigs.shape[1]:
        mat1 = est_sigs
        mat2 = true_sigs
    else:
        mat1 = true_sigs
        mat2 = est_sigs

    if dist == "cos":
        con_mat = cdist(mat1.T, mat2.T, "cosine")
    elif dist == "cor":
        con_mat = cdist(mat1.T, mat2.T, "correlation")

    row_ind, col_ind = linear_sum_assignment(con_mat)
    con_mat = 1 - con_mat  # Convert distance -> similarity

    idxPair = {}
    true_positives = 0
    for x, y in zip(row_ind, col_ind):
        idxPair[x] = y
        if con_mat[x, y] >= cutoff:
            true_positives += 1

    computedFalsePositives = mat1.shape[1] - true_positives
    computedFalseNegatives = computedFalsePositives

    if true_sigs.shape[1] >= est_sigs.shape[1]:
        baseFalsePositives = 0
        baseFalseNegatives = true_sigs.shape[1] - est_sigs.shape[1]
    else:
        baseFalsePositives = est_sigs.shape[1] - true_sigs.shape[1]
        baseFalseNegatives = 0

    false_positives = baseFalsePositives + computedFalsePositives
    false_negatives = baseFalseNegatives + computedFalseNegatives
    number_of_ground_truth_signatures = true_sigs.shape[1]
    number_of_detected_signature = est_sigs.shape[1]

    try:
        precision = round(true_positives / (true_positives + false_positives), 2)
        recall = round(true_positives / (true_positives + false_negatives), 2)
        f1_score = round(2 * precision * recall / (precision + recall), 2)
    except ZeroDivisionError:
        precision = 0
        recall = 0
        f1_score = 0

    return (
        number_of_ground_truth_signatures,
        number_of_detected_signature,
        true_positives,
        false_positives,
        false_negatives,
        precision,
        recall,
        f1_score,
        idxPair,
    )

In [48]:


def pam_cosine_clustering(data, n_clusters=None, random_seed=seed):
    """
    Performs PAM clustering using cosine distance on given signature data.

    Parameters:
    - data (np.ndarray or pd.DataFrame): Signature data, where each column is a signature vector.
    - n_clusters (int): Number of clusters to form.
    - random_seed (int): Seed for reproducibility.

    Returns:
    - clusters (list): List of clusters, where each cluster is a list of indices.
    - medoid_signatures (np.ndarray): Array of medoid signature vectors.
    """
    random.seed(random_seed)  # Set random seed for reproducibility

    # Ensure the input is a NumPy array
    if isinstance(data, pd.DataFrame):
        data = data.to_numpy()  # Convert DataFrame to NumPy array

    # Transpose the data to treat columns as signatures
    data = data.T  # Shape: (num_signatures, num_bins)
    num_signatures, num_bins = data.shape

    if n_clusters > num_signatures:
        raise ValueError(
            f"n_clusters ({n_clusters}) is greater than total signatures ({num_signatures})."
        )
    
    # Define the cosine distance metric
    def custom_cosine_distance(vec_a, vec_b):
        return cdist([vec_a], [vec_b], "cosine")[0, 0]

    metric = distance_metric(type_metric.USER_DEFINED, func=custom_cosine_distance)

    # Perform k-medoids clustering
    # Pick random distinct indices as initial medoids
    initial_medoids = random.sample(range(num_signatures), n_clusters)

    # Convert data to a list of lists (required by pyclustering)
    data_list = data.tolist()

    # Run k-medoids (PAM)
    kmedoids_instance = kmedoids(
        data=data_list,
        initial_index_medoids=initial_medoids,
        metric=metric
    )
    kmedoids_instance.process()

    # Retrieve clusters and medoids
    clusters = kmedoids_instance.get_clusters()  # List of cluster indices
    medoids = kmedoids_instance.get_medoids()    # Indices of medoids

    # Retrieve the actual medoid signature vectors
    medoid_signatures = np.array([data[idx] for idx in medoids])

    return clusters, medoid_signatures


In [49]:
# _______Signature matrices EXTRACTION from JSON_______

results_dir = "results"   # Path to your JSON file's folder
results_csv_dir = "results_csv" 

# Create the results_csv directory if it doesn't exist
os.makedirs(results_csv_dir, exist_ok=True)

# Ensure the s_15, s_8, and s_25 folders exist
for parent_folder in ["s_15", "s_8", "s_25"]:
    parent_folder_path = os.path.join(results_csv_dir, parent_folder)
    os.makedirs(parent_folder_path, exist_ok=True)

    # Create noise level subfolders inside each parent folder
    for noise_level in ["n_0.02", "n_0.04", "n_0.08"]:
        noise_folder_path = os.path.join(parent_folder_path, noise_level)
        os.makedirs(noise_folder_path, exist_ok=True)

# Process each JSON file in the results directory
for filename in os.listdir(results_dir):
    if filename.endswith(".json"):
        file_path = os.path.join(results_dir, filename)
        
        # Parse the JSON data
        parsed_data = load_data(file_path)

        # Extract the main folder name based on "s_15", "s_8", "s_25"
        if "s_15" in filename:
            parent_folder = "s_15"
        elif "s_8" in filename:
            parent_folder = "s_8"
        elif "s_25" in filename:
            parent_folder = "s_25"
        else:
            raise ValueError(f"Unknown parent folder for file: {filename}")

        # Extract the noise level from the filename
        if "n_0.02" in filename:
            noise_level = "n_0.02"
        elif "n_0.04" in filename:
            noise_level = "n_0.04"
        elif "n_0.08" in filename:
            noise_level = "n_0.08"
        else:
            raise ValueError(f"Unknown noise level for file: {filename}")

        # Create the parent folder and noise level folder
        parent_folder_path = os.path.join(results_csv_dir, parent_folder)
        noise_folder_path = os.path.join(parent_folder_path, noise_level)
        os.makedirs(noise_folder_path, exist_ok=True)

        # Get the subfolder name (without .json)
        subfolder_name = os.path.splitext(filename)[0]
        subfolder_name = subfolder_name[subfolder_name.find('_')+1:] if '_' in subfolder_name else subfolder_name
        subfolder_path = os.path.join(noise_folder_path, subfolder_name)
        
        # Create the subfolder
        os.makedirs(subfolder_path, exist_ok=True)

        # Create separate subfolders for Dw and Dkl
        dw_folder_path = os.path.join(subfolder_path, "Dw")
        dkl_folder_path = os.path.join(subfolder_path, "Dkl")
        os.makedirs(dw_folder_path, exist_ok=True)
        os.makedirs(dkl_folder_path, exist_ok=True)

        # Save all Dw matrices to CSV
        if 'all_Dw' in parsed_data:
            for i, dw_matrix in parsed_data['all_Dw'].items():
                dw_csv_path = os.path.join(dw_folder_path, f"Dw_{i}.csv")
                dw_matrix.to_csv(dw_csv_path, index=False, header=False)  # Exclude index and header

        # Save all Dkl matrices to CSV
        if 'all_Dkl' in parsed_data:
            for i, dkl_matrix in parsed_data['all_Dkl'].items():
                dkl_csv_path = os.path.join(dkl_folder_path, f"Dkl_{i}.csv")
                dkl_matrix.to_csv(dkl_csv_path, index=False, header=False)  # Exclude index and header


In [50]:
# _______COSMIC signatures exctraction_______
# Define the filename
filename = "Results_20250116_194705_overall_s_8_n_0.02__GRCh37_10a_56_10d_52_36_91_45_38.json"

# Step 1: Extract signature names after "GRCh37"
signature_portion = re.search(r"GRCh37_([\w\d_]+)", filename).group(1)  # Extracts "10a_56_10d_52_36_91_45_38"
signature_ids = signature_portion.split("_")  # Splits into ['10a', '56', '10d', '52', ...]

# Prepend "SBS" to each signature ID
signature_names = [f"SBS{id}" for id in signature_ids]
print("Extracted Signature Names:", signature_names)

# Step 2: Load the COSMIC signatures file
cosmic_signatures_path = "cosmic_signatures/COSMIC_v3.4_SBS_GRCh37.txt"
signatures_df = pd.read_csv(cosmic_signatures_path, sep="\t")

# Set the index if the "Type" column exists
if "Type" in signatures_df.columns:
    signatures_df.set_index("Type", inplace=True)

# Step 3: Filter the signatures corresponding to the extracted names
filtered_signatures = signatures_df[signature_names]
print("Filtered Signatures:\n", filtered_signatures)

# Convert the filtered signatures to a NumPy array (if needed)
true_signatures = filtered_signatures.to_numpy()
true_sig_names = filtered_signatures.columns


Extracted Signature Names: ['SBS10a', 'SBS56', 'SBS10d', 'SBS52', 'SBS36', 'SBS91', 'SBS45', 'SBS38']
Filtered Signatures:
                SBS10a     SBS56    SBS10d     SBS52     SBS36     SBS91  \
Type                                                                      
A[C>A]A  2.190170e-03  0.012597  0.010114  0.015196  0.025194  0.002945   
A[C>A]C  1.770137e-03  0.015697  0.018446  0.006538  0.008318  0.052997   
A[C>A]G  1.500120e-04  0.000206  0.000727  0.004139  0.002239  0.000204   
A[C>A]T  1.700132e-02  0.022995  0.014197  0.009238  0.017896  0.000131   
A[C>G]A  2.230000e-16  0.000418  0.000129  0.001730  0.001840  0.000243   
...               ...       ...       ...       ...       ...       ...   
T[T>C]T  3.250252e-03  0.000285  0.007555  0.002000  0.002799  0.001274   
T[T>G]A  2.690209e-03  0.009458  0.019898  0.001430  0.000788  0.005955   
T[T>G]C  2.230000e-16  0.000001  0.000738  0.001120  0.000744  0.000143   
T[T>G]G  2.160000e-05  0.000101  0.003148  0.001480